### This is the code for prompting a LLM 

Would like to use chatCPT4o but can also do other models found on Hugging Face like ChatGPT-3, but as OpenAI would not provide us with a key we decided to use transformer models that can be found on the Hugging Face libaries. 


In [ ]:
from transformers import pipeline
import json

# Loading the model from Hugging Face 
model_name = "bigscience/bloom"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name) 


# function for reading JSONL file 
def read_jsonl(file_path): 
    with open(file_path, "r", encoding="utf-8") as file: 
        lines = file.readlines()
        return [json.loads(line) for line in lines]
    

